<a href="https://colab.research.google.com/github/Ahayaat/ATM/blob/main/Rreserve_ROOM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langgraph langsmith langchain langchain_google_genai langchain_community

In [ ]:
!pip install python-decouple

In [ ]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Room Reservation Chatbot"

gemini_api_key = userdata.get('GEMINI_API_KEY')
DB_URI = userdata.get('DB_URI')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model='gemini-1.5-flash',
    max_retries=2,
    api_key=gemini_api_key,
)

llm.invoke("Hello Amala")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-3d67e971-0473-46c0-884c-73ce741e677f-0', usage_metadata={'input_tokens': 4, 'output_tokens': 10, 'total_tokens': 14, 'input_token_details': {'cache_read': 0}})

In [ ]:
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END, MessagesState
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, RemoveMessage
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
memory = MemorySaver()

In [ ]:
graph_builder = StateGraph(State)

In [ ]:
from random import randint

In [ ]:
def make_reservation(room_type: str, nights: int):
    """
    Constructs a reservation summary.

    Args:
        room_type: str - The type of room being reserved (e.g., single, double, suite).
        nights: int - The number of nights for the reservation.

    Returns:
        str - A summary of the reservation.
    """
    return f"Reservation confirmed: {room_type} room for {nights} night(s)."


In [ ]:
def check_room_availability(total_rooms: int, booked_rooms: int):
    """
    Checks the availability of rooms.

    Args:
        total_rooms: int - The total number of rooms in the hotel.
        booked_rooms: int - The number of rooms that are already booked.

    Returns:
        str - A message indicating the number of available rooms.
    """
    if booked_rooms > total_rooms:
        return "Error: Booked rooms cannot exceed total rooms."
    available_rooms = total_rooms - booked_rooms
    return f"{available_rooms} room(s) are available out of {total_rooms} total room(s)."


In [ ]:
def cancel_reservation(room_type: str, nights: int):
    """
    Cancels a reservation.

    Args:
        room_type: str - The type of room being reserved (e.g., single, double, suite).
        nights: int - The number of nights for the reservation.

    Returns:
        str - A message indicating the cancellation of the reservation.
    """
    return f"Reservation canceled: {room_type} room for {nights} night(s)."

In [ ]:
tools = [make_reservation, check_room_availability, cancel_reservation]

In [ ]:
llm_call_tools = llm.bind_tools(tools)

In [ ]:
def call_func(state: State):
  return{"messages": llm_call_tools.invoke(state["messages"])}

In [ ]:
graph_builder.add_node("call_func", call_func)
graph_builder.add_node("tools", ToolNode(tools))

graph_builder.add_edge(START, "call_func")
graph_builder.add_conditional_edges("call_func",tools_condition)
graph_builder.add_edge("tools", "call_func")

In [ ]:
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "1"}}

In [ ]:
sys_msg = SystemMessage(content="You are a helpful Room Reservation Expert who assists users with their queries.", name="ReservationBot")

In [ ]:
graph.stream({"messages": [sys_msg, HumanMessage(content="hello how ru")]}, config, stream_mode="updates")

<generator object Pregel.stream at 0x7c5e48284420>

In [ ]:
graph.invoke({"messages": [sys_msg, HumanMessage(content="How many rooms are available if there are 20 rooms and 7 are already booked?")]}, config)

{'messages': [SystemMessage(content='You are a helpful Room Reservation Expert who assists users with their queries.', additional_kwargs={}, response_metadata={}, name='ReservationBot', id='bb417159-ef9f-4cc7-b0fd-7878bbb3eac5'),
  HumanMessage(content='How many rooms are available if there are 20 rooms and 7 are already booked?', additional_kwargs={}, response_metadata={}, id='6e062337-447b-43e6-a7f0-24a752c61d32'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'check_room_availability', 'arguments': '{"total_rooms": 20.0, "booked_rooms": 7.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-e58cb9e4-e399-4458-9a9a-7f38bcd94d88-0', tool_calls=[{'name': 'check_room_availability', 'args': {'total_rooms': 20.0, 'booked_rooms': 7.0}, 'id': 'd4e12690-ed7d-4d30-907b-a1492a6acf49', 'type': 'tool_call'}], usage_metadata={'input_tokens': 272, 'output_tokens': 11, 'total_tokens': 2

In [ ]:
import asyncio

In [ ]:
while True:
    user_input = input("Write prompt: ")

    if user_input.lower() in ["quit", "exit", "q"]:
      print("Goodbye!")
      break
    async for event in graph.astream_events(
        {"messages": [HumanMessage(content=user_input)]},
        config,
        version="v2",
    ):
      if event["event"] == "on_chain_stream" and event["name"] == "call_func":
        print(event["data"]["chunk"]["messages"])

Write prompt: hi
content='Hello! How can I help you today?' additional_kwargs={} response_metadata={'safety_ratings': [], 'finish_reason': 'STOP'} id='run-e5f942ea-6b02-44c3-a983-675eef3e4f39' usage_metadata={'input_tokens': 346, 'output_tokens': 10, 'total_tokens': 356, 'input_token_details': {'cache_read': 0}}
Write prompt: room cancelation for 2 noght
content='I can help with that.  However, I need to know the room type to cancel the reservation.  Could you please tell me the type of room (single, double, suite, etc.)?' additional_kwargs={} response_metadata={'safety_ratings': [], 'finish_reason': 'STOP'} id='run-52d1cb23-51b8-4e80-b48d-90e23cc6638e' usage_metadata={'input_tokens': 365, 'output_tokens': 42, 'total_tokens': 407, 'input_token_details': {'cache_read': 0}}
Write prompt: double room
content='' additional_kwargs={'function_call': {'name': 'cancel_reservation', 'arguments': '{"room_type": "double", "nights": 2.0}'}} response_metadata={'finish_reason': 'STOP', 'safety_ratin